In [2]:
"""
Adaptive Learning Tutor - Multi-Agent System
5-Day AI Agents Intensive Course Capstone Project

This module implements a personalized education system using multiple AI agents
powered by Google's Gemini API.
"""

import os
import json
import uuid
from typing import Dict, List, Any, Optional
from dataclasses import dataclass, asdict
from datetime import datetime
from enum import Enum

# ADK imports - Following course patterns
from google.adk.agents import LlmAgent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import FunctionTool, google_search
from google.adk.tools.tool_context import ToolContext
from google.adk.models.google_llm import Gemini
from google.genai import types
from kaggle_secrets import UserSecretsClient

In [3]:
# ============================================================================
# CONFIGURATION
# ============================================================================

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

# Model configuration with retry (from Day 1)
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

MODEL_NAME = "gemini-2.5-flash-lite"

✅ Gemini API key setup complete.


In [4]:
# ============================================================================
# DATA MODELS
# ============================================================================

class DifficultyLevel(Enum):
    """Difficulty levels for content and assessments"""
    BEGINNER = "beginner"
    INTERMEDIATE = "intermediate"
    ADVANCED = "advanced"


class LearningStyle(Enum):
    """Student learning style preferences"""
    VISUAL = "visual"
    AUDITORY = "auditory"
    KINESTHETIC = "kinesthetic"
    READING_WRITING = "reading_writing"


@dataclass
class StudentProfile:
    """Represents a student's learning profile"""
    student_id: str
    name: str
    grade_level: int
    subject: str
    current_topic: Optional[str] = None
    knowledge_map: Dict[str, float] = None
    learning_style: LearningStyle = LearningStyle.VISUAL
    preferred_difficulty: DifficultyLevel = DifficultyLevel.BEGINNER
    session_count: int = 0
    created_at: str = None
    
    def __post_init__(self):
        if self.knowledge_map is None:
            self.knowledge_map = {}
        if self.created_at is None:
            self.created_at = datetime.now().isoformat()


@dataclass
class Question:
    """Represents an assessment question"""
    question_id: str
    question_text: str
    topic: str
    difficulty: DifficultyLevel
    correct_answer: str
    options: List[str]
    explanation: str


@dataclass
class AssessmentResult:
    """Results from an assessment"""
    student_id: str
    topic: str
    total_questions: int
    correct_answers: int
    proficiency_score: float
    knowledge_gaps: List[str]
    strengths: List[str]
    timestamp: str = None
    
    def __post_init__(self):
        if self.timestamp is None:
            self.timestamp = datetime.now().isoformat()

In [5]:
# ============================================================================
# MEMORY BANK 
# ============================================================================

class MemoryBank:
    """
    Session and state management.
    Uses InMemorySessionService pattern but simplified for demo.
    """
    
    def __init__(self):
        self.students: Dict[str, StudentProfile] = {}
        self.assessment_history: Dict[str, List[AssessmentResult]] = {}
    
    def save_student(self, profile: StudentProfile) -> None:
        """Save or update student profile"""
        self.students[profile.student_id] = profile
        print(f"[MemoryBank] Saved profile for: {profile.student_id}")
    
    def get_student(self, student_id: str) -> Optional[StudentProfile]:
        """Retrieve student profile"""
        return self.students.get(student_id)
    
    def save_assessment_result(self, result: AssessmentResult) -> None:
        """Save assessment results to history"""
        if result.student_id not in self.assessment_history:
            self.assessment_history[result.student_id] = []
        self.assessment_history[result.student_id].append(result)
        print(f"[MemoryBank] Saved assessment for: {result.student_id}")


In [6]:
# ============================================================================
# ASSESSMENT AGENT - Following Day 2 Tool Patterns
# ============================================================================

class AssessmentAgent:
    """
    Assessment agent using proper ADK patterns.
    Uses FunctionTool wrapper.
    """
    
    def __init__(self, memory_bank: MemoryBank):
        self.memory_bank = memory_bank
        
        # Create the LlmAgent with proper tools - Day 2 pattern
        self.agent = LlmAgent(
            model=Gemini(model=MODEL_NAME, retry_options=retry_config),
            name="AssessmentAgent",
            description="Generates questions and evaluates student responses",
            instruction="""You are a specialized assessment agent.
            
            Your tasks:
            1. Generate diagnostic questions for given topics and grade levels
            2. Evaluate student responses and calculate proficiency scores
            3. Identify knowledge gaps and strengths
            
            Use the provided tools to generate questions and evaluate responses.
            Always provide constructive feedback.""",
            tools=[
                FunctionTool(func=self._generate_questions_tool),
                FunctionTool(func=self._evaluate_responses_tool)
            ]
        )
        
        print("[AssessmentAgent] Initialized with LlmAgent")
    
    def _generate_questions_tool(
        self, 
        topic: str, 
        grade_level: int,
        num_questions: int = 5,
        difficulty: str = "beginner"
    ) -> str:
        """
        Tool function to generate assessment questions.
        Returns JSON string as expected by LLM.
        """
        print(f"[Tool] Generating {num_questions} questions for: {topic}")
        
        # In production, this would use the LLM to generate questions
        # For demo, return structured format
        questions_template = {
            "topic": topic,
            "grade_level": grade_level,
            "difficulty": difficulty,
            "questions": [
                {
                    "question_id": f"q_{i}",
                    "question_text": f"Question {i} about {topic}",
                    "options": ["A) Option 1", "B) Option 2", "C) Option 3", "D) Option 4"],
                    "correct_answer": "A",
                    "explanation": "Explanation here"
                }
                for i in range(num_questions)
            ]
        }
        
        return json.dumps(questions_template)
    
    def _evaluate_responses_tool(
        self,
        student_id: str,
        topic: str,
        responses: str  # JSON string of responses
    ) -> str:
        """
        Tool function to evaluate student responses.
        Returns evaluation as JSON string.
        """
        print(f"[Tool] Evaluating responses for: {student_id}")
        
        # Parse responses (in production, this would be more sophisticated)
        try:
            response_data = json.loads(responses)
            total = len(response_data.get("answers", []))
            correct = sum(1 for ans in response_data.get("answers", []) if ans.get("correct", False))
        except:
            total = 5
            correct = 3  # Mock data
        
        proficiency = correct / total if total > 0 else 0
        
        result = AssessmentResult(
            student_id=student_id,
            topic=topic,
            total_questions=total,
            correct_answers=correct,
            proficiency_score=proficiency,
            knowledge_gaps=["gap1", "gap2"] if proficiency < 0.7 else [],
            strengths=["strength1"] if proficiency > 0.6 else []
        )
        
        self.memory_bank.save_assessment_result(result)
        
        return json.dumps({
            "status": "success",
            "proficiency_score": proficiency,
            "correct_answers": correct,
            "total_questions": total,
            "feedback": "Good progress!" if proficiency > 0.7 else "Keep practicing!"
        })

In [ ]:
async def run_session(
    runner_instance: Runner, user_queries: list[str] | str, session_id: str = "default"
):
    """Helper function to run queries in a session and display responses."""
    print(f"\n### Session: {session_id}")

    # Create or retrieve session
    try:
        session = await session_service.create_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )
    except:
        session = await session_service.get_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )

    # Convert single query to list
    if isinstance(user_queries, str):
        user_queries = [user_queries]

    # Process each query
    for query in user_queries:
        print(f"\nUser > {query}")
        query_content = types.Content(role="user", parts=[types.Part(text=query)])

        # Stream agent response
        async for event in runner_instance.run_async(
            user_id=USER_ID, session_id=session.id, new_message=query_content
        ):
            if event.is_final_response() and event.content and event.content.parts:
                text = event.content.parts[0].text
                if text and text != "None":
                    print(f"Model: > {text}")

In [7]:
# ============================================================================
# CONTENT GENERATION AGENT - Using Google Search Tool
# ============================================================================

class ContentGenerationAgent:
    """
    Content generation agent with Google Search integration.
    Follows Day 1 patterns for tool integration.
    """
    
    def __init__(self, memory_bank: MemoryBank):
        self.memory_bank = memory_bank
        
        self.agent = LlmAgent(
            model=Gemini(model=MODEL_NAME, retry_options=retry_config),
            name="ContentGenerationAgent",
            description="Generates personalized learning content",
            instruction="""
            You are a content generation specialist.

            Your tasks:
            1. Create tailored lesson content based on student level and learning style
            2. Generate practice problems with solutions
            3. Find real-world examples using Google Search when needed
            4. Adapt explanations to different learning styles
            
            Always make content engaging and appropriate for the grade level.""",
            tools=[
                FunctionTool(func=self._generate_lesson_tool),
                google_search  # Built-in tool from Day 1
            ]
        )
        
        print("[ContentGenerationAgent] Initialized with Google Search")
    
    def _generate_lesson_tool(
        self,
        topic: str,
        difficulty: str,
        learning_style: str,
        grade_level: int
    ) -> str:
        """
        Tool to generate lesson content.
        Returns lesson as JSON string.
        """
        print(f"[Tool] Generating lesson: {topic} ({difficulty})")
        
        lesson = {
            "topic": topic,
            "difficulty": difficulty,
            "learning_style": learning_style,
            "explanation": f"Comprehensive explanation of {topic} for grade {grade_level}",
            "examples": [
                "Example 1: Basic concept",
                "Example 2: Applied concept",
                "Example 3: Real-world application"
            ],
            "practice_problems": [
                {
                    "problem": "Practice problem 1",
                    "solution": "Step-by-step solution",
                    "answer": "Final answer"
                }
            ],
            "key_takeaways": [
                "Key point 1",
                "Key point 2",
                "Key point 3"
            ]
        }
        
        return json.dumps(lesson)


In [8]:
# ============================================================================
# ORCHESTRATOR - Multi-Agent Coordination (Day 1 Patterns)
# ============================================================================

class LearningTutorOrchestrator:
    """
    Main orchestrator using Runner pattern.
    Coordinates multiple agents in sequential and parallel flows.
    """
    
    def __init__(self):
        self.memory_bank = MemoryBank()
        self.session_service = InMemorySessionService()
        
        # Initialize agents
        self.assessment_agent = AssessmentAgent(self.memory_bank)
        self.content_agent = ContentGenerationAgent(self.memory_bank)
        
        # App configuration
        self.app_name = "adaptive_learning_tutor"
        self.user_id = "default_user"
        
        print("[Orchestrator] Initialized with ADK Runner pattern")
    
    def onboard_student(
        self,
        student_id: str,
        name: str,
        grade_level: int,
        subject: str
    ) -> StudentProfile:
        """Create new student profile"""
        profile = StudentProfile(
            student_id=student_id,
            name=name,
            grade_level=grade_level,
            subject=subject
        )
        self.memory_bank.save_student(profile)
        print(f"[Orchestrator] Onboarded: {name}")
        return profile
    
    async def run_initial_assessment(
        self,
        student_id: str,
        topic: str
    ):
        """
        PHASE 1 - SEQUENTIAL FLOW using Runner
        """
        print(f"\n{'='*60}")
        print("PHASE 1: INITIAL ASSESSMENT (Sequential)")
        print(f"{'='*60}\n")
        
        student = self.memory_bank.get_student(student_id)
        if not student:
            raise ValueError(f"Student {student_id} not found")
        
        # Create session 
        session_id = f"assessment_{uuid.uuid4().hex[:8]}"
        await self.session_service.create_session(
            app_name=self.app_name,
            user_id=self.user_id,
            session_id=session_id
        )
        
        # Runner
        runner = Runner(
            agent=self.assessment_agent.agent,
            app_name=self.app_name,
            session_service=self.session_service
        )
        
        # Query
        query = f"""Generate 5 diagnostic questions for:
        Topic: {topic}
        Grade Level: {student.grade_level}
        Difficulty: {student.preferred_difficulty.value}
        
        Return as structured JSON."""
        
        query_content = types.Content(
            role="user",
            parts=[types.Part(text=query)]
        )
        
        print(f"Generating questions for: {topic}")
        
        # Run agent 
        async for event in runner.run_async(
            user_id=self.user_id,
            session_id=session_id,
            new_message=query_content
        ):

            # ----------- HANDLE FINAL RESPONSE -----------
            if event.is_final_response():

                # If function_call (common!)
                if getattr(event, "function_call", None):
                    print("\n[Assessment] Model returned FUNCTION CALL instead of text")
                    print(event.function_call)
                    continue

                content = getattr(event, "content", None)

                if not content:
                    print("[Warning] event.content is None")
                    continue

                # If content.text exists
                if hasattr(content, "text") and content.text:
                    print("\n[Assessment] Generated questions (text)")
                    print(content.text[:200] + "...")
                    continue

                # If content.parts exists
                parts = getattr(content, "parts", None)

                if parts:
                    for part in parts:
                        if hasattr(part, "text") and part.text:
                            print("\n[Assessment] Generated questions (parts)")
                            print(part.text[:200] + "...")
                    continue

                # Nothing usable
                print("[Warning] No usable text, parts, or function_call in final response")
                print(event)
        
        print("\n✅ Assessment phase complete")


In [17]:

# ============================================================================
# MAIN EXECUTION
# ============================================================================

async def main():
    """Demonstrate the Adaptive Learning ADK-based system"""
    print("\n" + "="*60)
    print("ADAPTIVE LEARNING TUTOR")
    print("="*60 + "\n")
    
    # Initialize orchestrator
    orchestrator = LearningTutorOrchestrator()
    
    # Onboard student
    student = orchestrator.onboard_student(
        student_id="student_001",
        name="Yuvraj Singh Kiraula",
        grade_level=8,
        subject="Mathematics"
    )
    
    print(f"\n✅ Student Profile:")
    print(f"   Name: {student.name}")
    print(f"   Grade: {student.grade_level}")
    print(f"   Subject: {student.subject}\n")
    
    # Run assessment
    # await orchestrator.run_initial_assessment(
    #     student_id=student.student_id,
    #     topic="algebra"
    # )

    await run_initial_assessment_safe(orchestrator, "student_001", "Mathematics")
    
    print("\n" + "="*60)
    print("✅ DEMO COMPLETE - ADK Integration Successful!")
    print("="*60 + "\n")


In [18]:
await main()


ADAPTIVE LEARNING TUTOR

[AssessmentAgent] Initialized with LlmAgent
[ContentGenerationAgent] Initialized with Google Search
[Orchestrator] Initialized with ADK Runner pattern
[MemoryBank] Saved profile for: student_001
[Orchestrator] Onboarded: Yuvraj Singh Kiraula

✅ Student Profile:
   Name: Yuvraj Singh Kiraula
   Grade: 8
   Subject: Mathematics

Generating questions for: Mathematics (session=assessment_d03cb51b)


[Tool] Generating 5 questions for: Mathematics


ERROR:asyncio:Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7ea679eadb50>


[Warning] attempt 1 failed: No usable content returned by model
Generating questions for: Mathematics (session=assessment_8f0bfe04)


[Tool] Generating 5 questions for: Mathematics
[Warning] attempt 2 failed: No usable content returned by model
Generating questions for: Mathematics (session=assessment_b88635c2)


[Tool] Generating 5 questions for: Mathematics
[Warning] attempt 3 failed: No usable content returned by model


RuntimeError: No usable content returned by model